In [7]:
import pandas as pd
import numpy as np
import bisect
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
df = pd.read_csv("df_ff_factors.csv")
# df = df.drop(columns=["Unnamed: 0", "crsp_fundno","crsp_portno"])
# df = df.sort_values(by='date')
# df.columns

In [68]:
df.describe()

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,crsp_fundno,crsp_portno,date,mth_return,exp_ratio,turn_ratio,normalised_flow,gdp_to_debt_ratio,gdp_growth_rate,unm_rate,...,hml,rmw,cma,rf,umd,excess_return,rolling_sharpe,mkt_return,rolling_alpha,shortrun_momentum
count,54718.000000,5.471800e+04,54718,54718.000000,54718.000000,54718.000000,5.471200e+04,54718.000000,54718.000000,54718.000000,...,54718.000000,54718.000000,54718.000000,54718.000000,54718.000000,54718.000000,54718.000000,54718.000000,54718.000000,54718.000000
mean,25847.143774,1.025033e+06,2014-04-26 16:18:33.724916992,0.010032,0.013245,1.030010,NaN,95.332335,2.151457,5.734164,...,-0.000107,0.003847,0.001288,0.001162,0.001407,0.010020,0.246439,0.012268,-0.001697,-0.002178
min,2756.000000,1.000182e+06,1993-08-31 00:00:00,-0.535195,0.000000,0.000000,-inf,54.032000,-2.576500,3.400000,...,-0.138800,-0.186500,-0.072000,0.000000,-0.343000,-0.535208,-25.645915,-0.264049,-0.207237,-0.235318
25%,12043.000000,1.022836e+06,2009-06-30 00:00:00,-0.024254,0.008600,0.280000,-6.962523e-02,80.280000,1.819500,4.000000,...,-0.018300,-0.011000,-0.012300,0.000000,-0.020200,-0.024279,-0.005585,-0.020737,-0.007262,-0.012086
50%,19869.000000,1.024864e+06,2015-01-30 00:00:00,0.013020,0.012400,0.500000,-2.857526e-03,100.973000,2.457600,5.100000,...,-0.002000,0.004000,-0.000600,0.000300,0.004400,0.013005,0.230487,0.018009,-0.000963,-0.001809
75%,30567.000000,1.025998e+06,2020-01-31 00:00:00,0.048310,0.017800,1.040000,4.954032e-02,106.886000,2.945600,6.700000,...,0.015900,0.015800,0.012900,0.001800,0.027400,0.048297,0.494137,0.049571,0.004743,0.008017
max,95211.000000,1.051169e+06,2024-07-31 00:00:00,0.631549,0.037900,24.130000,inf,132.815000,5.800200,14.700000,...,0.128000,0.130700,0.090700,0.005600,0.182000,0.631548,138.764929,0.249812,0.329532,0.230276
std,20021.936361,6.798237e+03,NaN,0.065036,0.006393,1.888697,NaN,21.748174,1.897339,2.069179,...,0.034398,0.023506,0.021984,0.001469,0.045830,0.065036,0.801358,0.059404,0.015841,0.022838


In [2]:
# Generate Lagged Dataset
def create_lagged_dataset(dataset, lag, target_var):
    lagged_dataset = dataset.copy()
    columns_list = list(lagged_dataset.columns)
    data_join = {}
    for column in columns_list:
        if column == target_var:
            data_join[column] = lagged_dataset[column]
        for n in range(1,lag+1):
            data_join[F'{column}_L{n}'] = lagged_dataset[column].shift(n)
    lagged_dataset = pd.concat(data_join.values(), axis=1, ignore_index = True)
    lagged_dataset.columns = data_join.keys()
    return lagged_dataset.dropna()

# Generate Stepped Dataset for Training
## Steps is the number of months ahead that we are forecasting, e.g. step=2 is 2 months ahead.
## Note step=1 results in no change to dataset, i.e. use generated lagged variables to forecast current. 
def create_stepped_dataset(dataset, step, target_var):
    
    shifted_dataset = dataset.copy()
    y = shifted_dataset[[target_var]].shift(-step+1)
    if step != 1:
        X = shifted_dataset.iloc[:-step+1, :] # remove the last few rows
    else:
        X = shifted_dataset
    return X.drop(target_var, axis = 1), y.dropna()

x = create_lagged_dataset(df, lag = 1, target_var='rolling_sharpe')
test_step = create_stepped_dataset(x, step=1, target_var='rolling_sharpe')[0]

In [3]:
import numpy as np
corr = np.corrcoef(df['rolling_alpha'], df['rolling_sharpe'])
corr # 0.204 --> can use!

array([[1.        , 0.20433687],
       [0.20433687, 1.        ]])

3-Factor Model: mktrf, SMB, HML
4-Factor Model: mktrf, SMB, HML, UMD
5-Factor Model: mktrf, SMB, HML, RMW, CMA

In [221]:
df_3_factor = df.drop(columns=['mkt_return','umd','cma','rmw','mth_return','rf'])
df_3_factor = df[['date','mth_return','exp_ratio','turn_ratio','mktrf', 'smb', 'hml']]
df_4_factor = df.drop(columns=['mkt_return','cma','rmw','mth_return','rf'])
df_5_factor = df.drop(columns=['mkt_return','umd','mth_return','rf'])
df_4_factor

,date,exp_ratio,turn_ratio,normalised_flow,gdp_to_debt_ratio,gdp_growth_rate,unm_rate,infl_rate,mktrf,smb,hml,umd,excess_return,rolling_sharpe,rolling_alpha,shortrun_momentum
43631,1993-08-31,0.0162,0.15,0.009529,64.101,3.5225,6.8,2.90,0.0371,0.0010,0.0013,0.0265,0.068910,0.937887,0.012866,0.012866
43632,1993-09-30,0.0162,0.15,0.014881,64.101,3.5225,6.7,2.90,-0.0012,0.0298,-0.0031,0.0337,-0.004108,0.639835,-0.031091,-0.009113
43633,1993-10-29,0.0162,0.15,0.030641,64.669,3.5225,6.8,2.90,0.0141,0.0189,-0.0276,-0.0272,0.040962,0.882807,0.019375,0.000383
13747,1993-10-29,0.0188,0.77,0.070030,64.669,3.5225,6.8,2.90,0.0141,0.0189,-0.0276,-0.0272,0.034690,1.658248,0.013103,0.013103
13748,1993-11-30,0.0188,0.77,0.000259,64.669,3.5225,6.6,2.90,-0.0189,-0.0127,-0.0074,-0.0474,-0.075999,-0.155000,-0.065168,-0.026033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031,2024-06-28,0.0217,0.23,-0.650182,120.040,2.5427,4.1,3.35,0.0070,-0.0568,-0.0247,0.0508,0.078518,0.467504,-0.012479,-0.025985
1960,2024-07-31,0.0077,0.31,-0.022800,120.731,2.5427,4.2,3.35,0.0070,-0.0568,-0.0247,0.0508,0.028784,0.199592,-0.022571,-0.024241
2113,2024-07-31,0.0076,0.31,-0.402964,120.731,2.5427,4.2,3.35,0.0070,-0.0568,-0.0247,0.0508,0.028779,0.199688,-0.022571,-0.024228
1842,2024-07-31,0.0102,0.31,-0.032107,120.731,2.5427,4.2,3.35,0.0070,-0.0568,-0.0247,0.0508,0.028569,0.195258,-0.022790,-0.024451


In [16]:

# Naive Model
def create_naive(dataset, step, target_var):
    naive = dataset.copy()
    naive['naive'] = naive.groupby("crsp_fundno")[target_var].shift(step)
    return naive

# Generates next date
def generate_next_date(list_of_dates, date):
    return list_of_dates[list_of_dates > date].min()

# Training Cycle
def overall_function(dataset, outcome):
    dataset['date'] = pd.to_datetime(dataset['date']) # converting to date format
    dataset = dataset.sort_values(by='date')
    dataset = dataset.replace(np.inf, 0)
    dataset = dataset.replace(-np.inf, 0)
    df_3_factor = dataset.drop(columns=['mkt_return','umd','cma','rmw','mth_return','rf'])
    df_4_factor = dataset.drop(columns=['mkt_return','cma','rmw','mth_return','rf'])
    df_5_factor = dataset.drop(columns=['mkt_return','umd','mth_return','rf'])
    
    X_dataset_3f, y_dataset_3f = create_stepped_dataset(create_lagged_dataset(df_3_factor, lag=1,target_var=outcome),step=1,target_var=outcome)
    X_dataset_4f, y_dataset_4f = create_stepped_dataset(create_lagged_dataset(df_4_factor, lag=1,target_var=outcome),step=1,target_var=outcome)
    X_dataset_5f, y_dataset_5f = create_stepped_dataset(create_lagged_dataset(df_5_factor, lag=1,target_var=outcome),step=1,target_var=outcome)

    # 3-Factor model
    X_3f = X_dataset_3f.copy()
    X_4f = X_dataset_4f.copy()
    X_5f = X_dataset_5f.copy()
    
    # Model 1: Naive Model
    # y_pred_df.at[window_end+1, 'naive'] = naive_fxn.loc[window_end+1][0]
    
    # list_of_dates = dataset['date'].unique()
    list_of_dates = pd.to_datetime(dataset['date'])
    
    percentile_70 = list_of_dates.quantile(0.7)
    train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
    
    # window_end = generate_next_date(list_of_dates, window_start)
    # return(list_of_dates)
    df_end = list_of_dates.max()
    # return(df_end)
    results = []
    
    while train_end != df_end:
        
        test_date = generate_next_date(list_of_dates, train_end)
        # DATASET 1: data processing for 3f dataset
        
        X_train_3f = X_3f[X_3f['date_L1'] <= train_end]
        X_test_3f = X_3f[X_3f['date_L1'] == test_date]
        
        y_3f = y_dataset_3f.copy()
        y_train_3f = y_3f[:len(X_train_3f)]
        y_test_3f = y_3f[len(X_train_3f):(len(X_train_3f)+len(X_test_3f))]
        X_train_3f = X_train_3f.drop(columns='date_L1')
        X_test_3f = X_test_3f.drop(columns='date_L1')
        scaler = StandardScaler()
        X_train_3f = scaler.fit_transform(X_train_3f)
        X_test_3f = scaler.transform(X_test_3f)
        # return(len(X_train_3f))
        
        # For all models using all variables, I use dataframe and not array
        X_train_3f = pd.DataFrame(X_train_3f)
        X_train_3f.columns = [col for col in X_3f.columns if col != 'date_L1']
        X_test_3f = pd.DataFrame(X_test_3f)
        X_test_3f.columns = [col for col in X_3f.columns if col != 'date_L1']
        
        
        ## DATASET 2: data processing for 4f dataset
        X_train_4f = X_4f[X_4f['date_L1'] < train_end]
        y_4f = y_dataset_4f.copy()
        y_train_4f = y_4f[:len(X_train_4f)]
        X_test_4f = X_4f[X_4f['date_L1'] == test_date]
        X_train_4f = X_train_4f.drop(columns='date_L1')
        X_test_4f = X_test_4f.drop(columns='date_L1')
        scaler = StandardScaler()
        X_train_4f = scaler.fit_transform(X_train_4f)
        X_test_4f = scaler.transform(X_test_4f)
        # y_test_4f = y_4f[len(X_train_4f):(len(X_train_4f)+len(X_test_4f))]
        
        X_train_4f = pd.DataFrame(X_train_4f)
        X_train_4f.columns = [col for col in X_4f.columns if col != 'date_L1']
        X_test_4f = pd.DataFrame(X_test_4f)
        X_test_4f.columns = [col for col in X_4f.columns if col != 'date_L1']
        
        
        ## DATASET 3: data processing for 5f dataset
            
        X_train_5f = X_5f[X_5f['date_L1'] < train_end]
        y_5f = y_dataset_5f.copy()
        y_train_5f = y_5f[:len(X_train_5f)]
        X_test_5f = X_5f[X_5f['date_L1'] == test_date]
        X_train_5f = X_train_5f.drop(columns='date_L1')
        X_test_5f = X_test_5f.drop(columns='date_L1')
        scaler = StandardScaler()
        X_train_5f = scaler.fit_transform(X_train_5f)
        X_test_5f = scaler.transform(X_test_5f)
        # y_test_5f = y_5f[len(X_train_5f):(len(X_train_5f)+len(X_test_5f))]
        X_train_5f = pd.DataFrame(X_train_5f)
        X_train_5f.columns = [col for col in X_5f.columns if col != 'date_L1']
        X_test_5f = pd.DataFrame(X_test_5f)
        X_test_5f.columns = [col for col in X_5f.columns if col != 'date_L1']
        
        # Model 2: Linear Regression
        lm_model = LinearRegression()
        # return(y_train_3f.values)
        lm_model.fit(X_train_3f, y_train_3f.values.ravel())
        print('3-Factor for lm_model Fitted')
        y_pred_3f = lm_model.predict(X_test_3f)
        # return(len(y_pred_3f))
        lm_model.fit(X_train_4f, y_train_4f.values.ravel())
        print('4-Factor for lm_model Fitted')
        y_pred_4f = lm_model.predict(X_test_4f)
        
        lm_model.fit(X_train_5f, y_train_5f.values.ravel())
        print('5-Factor for lm_model Fitted')
        y_pred_5f = lm_model.predict(X_test_5f)
        
        df_in_loop = y_test_3f.copy()
        df_in_loop['3f_lm'] = y_pred_3f
        df_in_loop['4f_lm'] = y_pred_4f
        df_in_loop['5f_lm'] = y_pred_5f
        
        # Model 3: Feedforward Neural Network
        # model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=200, random_state=42)
        # # return(y_train_3f.values)
        # model.fit(X_train_3f, y_train_3f.values.ravel())
        # print('3-Factor for mlp_model Fitted')
        # y_pred_3f = model.predict(X_test_3f)
        # # return(len(y_pred_3f))
        # model.fit(X_train_4f, y_train_4f.values.ravel())
        # print('4-Factor for mlp_model Fitted')
        # y_pred_4f = model.predict(X_test_4f)
        
        # model.fit(X_train_5f, y_train_5f.values.ravel())
        # print('5-Factor for mlp_model Fitted')
        # y_pred_5f = model.predict(X_test_5f)
        
        # # df_in_loop = y_test_3f.copy()
        # df_in_loop['3f_ffnn'] = y_pred_3f
        # df_in_loop['4f_ffnn'] = y_pred_4f
        # df_in_loop['5f_ffnn'] = y_pred_5f
        # return(df_in_loop)
        
        # Model 4: Basic RNN
        
        # Model 5: Random Forest Regression
        model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

        # Train and predict for 3-factor model
        model.fit(X_train_3f, y_train_3f.values.ravel())
        print('3-Factor for rf_model Fitted')
        y_pred_3f = model.predict(X_test_3f)

        # Train and predict for 4-factor model
        model.fit(X_train_4f, y_train_4f.values.ravel())
        print('4-Factor for rf_model Fitted')
        y_pred_4f = model.predict(X_test_4f)

        # Train and predict for 5-factor model
        model.fit(X_train_5f, y_train_5f.values.ravel())
        print('5-Factor for rf_model Fitted')
        y_pred_5f = model.predict(X_test_5f)

        # Store predictions in df_in_loop
        df_in_loop['3f_rf'] = y_pred_3f
        df_in_loop['4f_rf'] = y_pred_4f
        df_in_loop['5f_rf'] = y_pred_5f
        # Add results into loop
        results.append(df_in_loop)
        train_end = test_date
        num_remaining_dates = len(list(set(date for date in list_of_dates if date > test_date)))
        print(f'{num_remaining_dates} dates remaining')
    combined_df = pd.concat(results, ignore_index=True)
    return combined_df

In [67]:
# create_naive(dataset=df,step=1,target_var='rolling_sharpe')
# df = df.drop(columns=["Unnamed: 0"])
c = create_naive(dataset=df,step=1,target_var='rolling_sharpe')
c = c.sort_values(by='date')
c

list_of_dates = pd.to_datetime(c['date'])
    
percentile_70 = list_of_dates.quantile(0.7)
train_end = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
d = c[c['date']>train_end]
d['mae_naive'] = abs(d['naive']-d['rolling_sharpe'])
mae_naive = d['mae_naive'].dropna().mean()
mae_naive

C:\Users\wjlwi\AppData\Local\Temp\ipykernel_34932\1903730267.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['mae_naive'] = abs(d['naive']-d['rolling_sharpe'])


0.1222434491884236

In [10]:
# x = overall_function(dataset = df, outcome = 'rolling_sharpe')
y = overall_function(dataset = df, outcome = 'rolling_sharpe')

3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
64 dates remaining
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
63 dates remaining
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
62 dates remaining
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
61 dates remaining
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
60 dates remaining
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
59 dates remaining
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
58 dates remaining
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
57 dates remaining
3-Factor Fitted
4-Factor Fitted
5-Factor Fitted
3-Factor Fitted
4-Factor Fitted


In [62]:
# y.to_csv('lm_rf.csv')
# x.to_csv('test.csv')
x = pd.read_csv('mlp.csv')
x['3f_mae'] = abs(x['3f_ffnn']-x['rolling_sharpe'])
x['4f_mae'] = abs(x['4f_ffnn']-x['rolling_sharpe'])
x['5f_mae'] = abs(x['5f_ffnn']-x['rolling_sharpe'])
mae_3f = x['3f_mae'].mean()

mae_4f = x['4f_mae'].mean()
mae_5f = x['5f_mae'].mean()
mae_4f/0.1282

1.917557219184085

In [65]:
x = pd.read_csv('lm_rf.csv')
x['3f_mae'] = abs(x['3f_rf']-x['rolling_sharpe'])
x['4f_mae'] = abs(x['4f_rf']-x['rolling_sharpe'])
x['5f_mae'] = abs(x['5f_rf']-x['rolling_sharpe'])
mae_3f = x['3f_mae'].mean()

mae_4f = x['4f_mae'].mean()
mae_5f = x['5f_mae'].mean()
mae_5f/0.1282

2.074302476855535

Random Test Codes

In [222]:
# from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
# from sklearn.metrics import r2_score
# X, y = create_stepped_dataset(create_lagged_dataset(df_3_factor, lag=1,target_var='mth_return'),step=1,target_var='mth_return')
# X = X.replace(np.inf, 0)
# X = X.replace(-np.inf, 0)
# # X

# list_of_dates = df['date'].unique()
# list_of_dates = pd.to_datetime(df['date']) # Converting to date format

# percentile_70 = list_of_dates.quantile(0.7)
# med_date = list_of_dates.loc[(list_of_dates - percentile_70).abs().idxmin()]
# next_date = list_of_dates[list_of_dates > med_date].min()

# X['date_L1'] = pd.to_datetime(X['date_L1'])
# X_train_3f = X[X['date_L1'] < med_date]
# y_train_3f = y[:len(X_train_3f)]
# X_test_3f = X[X['date_L1'] == next_date]
# y_test_3f = y[len(X_train_3f):(len(X_train_3f)+len(X_test_3f))]
# final = y_test_3f.copy()
# X_train_3f = X_train_3f.drop(columns='date_L1')
# X_test_3f = X_test_3f.drop(columns='date_L1')

# scaler = StandardScaler()

# X_train_3f = scaler.fit_transform(X_train_3f)
# X_test_3f = scaler.transform(X_test_3f)

# model = LinearRegression()
# model.fit(X_train_3f, y_train_3f)

# y_pred = model.predict(X_test_3f)

# model2 = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=200)
# model2.fit(X_train_3f, y_train_3f)

# y_pred2 = model2.predict(X_test_3f)

# tscv = TimeSeriesSplit(n_splits = 5)
# ridge_cv = RidgeCV(cv = tscv)
# ridge_cv.fit(X_train_3f, y_train_3f)

# model3 = Ridge(alpha = ridge_cv.alpha_)
# model3.fit(X_train_3f, y_train_3f)
# y_pred3 = model3.predict(X_test_3f)

# lasso_cv = LassoCV(cv = tscv, random_state = 18, max_iter = 100000)
# lasso_cv.fit(X_train_3f, y_train_3f)

# # Create the Lasso model with the optimal alpha value
# lasso_model = Lasso(alpha = lasso_cv.alpha_)
# lasso_model.fit(X_train_3f, y_train_3f)
# y_pred4 = lasso_model.predict(X_test_3f)

# elasticnet_cv = ElasticNetCV(cv = tscv, max_iter = 100000)
# elasticnet_cv.fit(X_train_3f, y_train_3f)
# elasticnet_model = ElasticNet(alpha = elasticnet_cv.alpha_, l1_ratio = elasticnet_cv.l1_ratio_)
# elasticnet_model.fit(X_train_3f, y_train_3f)
        
# y_pred5 = elasticnet_cv.predict(X_test_3f)

# final['linear'] = y_pred
# final['mlp'] = y_pred2
# # final['ridge'] = y_pred3
# # final['lasso'] = y_pred4
# # final['elasticnet'] = y_pred5

# final
# X_test_3f
# y_pred - y_test_3f
# sum(abs((y_pred-y_test_3f)/y_test_3f))/len(y_test_3f)

C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\wjlwi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passe

,mth_return,linear,mlp,ridge,lasso,elasticnet
10946,0.065153,0.021697,0.028095,0.021687,0.020224,0.020162
39688,0.061436,0.025942,0.028074,0.025939,0.025101,0.025072
5363,0.016949,0.016051,0.011053,0.016035,0.014860,0.014763
47379,0.060768,0.019023,0.010773,0.019012,0.018111,0.018036
28685,0.039707,0.019021,0.012601,0.019010,0.018229,0.018155
...,...,...,...,...,...,...
47060,0.062213,0.026357,0.028813,0.026354,0.025445,0.025418
35538,0.063594,0.018598,0.017622,0.018585,0.017400,0.017319
38224,0.060625,0.027637,0.031756,0.027635,0.026746,0.026727
16851,0.013965,0.025182,0.028781,0.025177,0.024109,0.024072


In [229]:
from sklearn.metrics import r2_score
r2_linear = r2_score(final['mth_return'], final['linear'])
r2_mlp = r2_score(final['mth_return'], final['mlp'])
# r2_ridge = r2_score(final['mth_return'], final['ridge'])
# r2_lasso = r2_score(final['mth_return'], final['lasso'])
# r2_elasticnet = r2_score(final['mth_return'], final['elasticnet'])
r2_mlp

-1.672832028647084